In [3]:
import cv2
import webbrowser
import time
import datetime

cap = cv2.VideoCapture(0)

# if not cap.isOpened():
#     print("Error: Camera could not be opened.")
#     exit()

# Background subtractor
backSub = cv2.createBackgroundSubtractorMOG2()

# Setting of roi, this program provide 4 squares to have instructions, such as screen shot, open up website and exit the program.
roi1 = (100, 100, 200, 200)
roi2 = (400, 100, 200, 200)
roi3 = (700, 100, 200, 200)  
roi4 = (1000, 100, 200, 200)  

time_in_roi1 = 0
time_in_roi2 = 0
time_in_roi3 = 0
time_in_roi4 = 0

threshold = 20
time_threshold = 20
url1 = "https://nccu.edu.tw/indexs.htmlm"
url2 = "https://i.nccu.edu.tw/Login.aspx?ReturnUrl=%2f"

while True:
    ret, frame = cap.read()
    if not ret:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Apply background subtraction
    fgMask = backSub.apply(frame)

    # Check for presence in each ROI and increment time if presence is detected
    time_in_roi1 += (fgMask[roi1[1]:roi1[1]+roi1[3], roi1[0]:roi1[0]+roi1[2]].mean() > threshold)
    time_in_roi2 += (fgMask[roi2[1]:roi2[1]+roi2[3], roi2[0]:roi2[0]+roi2[2]].mean() > threshold)
    time_in_roi3 += (fgMask[roi3[1]:roi3[1]+roi3[3], roi3[0]:roi3[0]+roi3[2]].mean() > threshold)
    time_in_roi4 += (fgMask[roi4[1]:roi4[1]+roi4[3], roi4[0]:roi4[0]+roi4[2]].mean() > threshold)

    # Draw ROI
    cv2.rectangle(frame, (roi1[0], roi1[1]), (roi1[0]+roi1[2], roi1[1]+roi1[3]), (255, 0, 0), 2)
    cv2.putText(frame, "Open Mail", (roi1[0], roi1[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    cv2.rectangle(frame, (roi2[0], roi2[1]), (roi2[0]+roi2[2], roi2[1]+roi2[3]), (0, 255, 0), 2)
    cv2.putText(frame, "Moodle", (roi2[0], roi2[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.rectangle(frame, (roi3[0], roi3[1]), (roi3[0]+roi3[2], roi3[1]+roi3[3]), (0, 0, 255), 2)  # Third ROI
    cv2.putText(frame, "Take Screenshot", (roi3[0], roi3[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.rectangle(frame, (roi4[0], roi4[1]), (roi4[0]+roi4[2], roi4[1]+roi4[3]), (255, 255, 0), 2)  # Fourth ROI
    cv2.putText(frame, "Exit", (roi4[0], roi4[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # Show the frame
    cv2.imshow('Frame', frame)

    # Open website
    if time_in_roi1 > time_threshold:
        webbrowser.open(url1)
        time_in_roi1 = 0 
        continue

    if time_in_roi2 > time_threshold:
        webbrowser.open(url2)
        time_in_roi2 = 0  
        continue

    # Screen Shot
    if time_in_roi3 > time_threshold:
        screenshot_filename = f"screenshot_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
        cv2.imwrite(screenshot_filename, frame)
        print(f"Screenshot saved as {screenshot_filename}")
        time_in_roi3 = 0 

    # Exit
    if time_in_roi4 > 100:
        break 

cap.release()
cv2.destroyAllWindows()
